# With statement

<font color='#bb9af7'>with as</font> statement is a powerful tool to manage external resources in your programs. Uses <i>context managers</i> to enter and exit these streams e.g files & communication.

## Syntax

<font color='#bb9af7'>with</font> statement before the <i>context manager</i>, and optionally the <font color='#bb9af7'>as</font> keyword to alias an object generated as <i>resource</i>.

In [ ]:
import dotenv
import os

FILENAME = 'with.csv'

dotenv.load_dotenv()
folder = os.getenv('TEMP_FOLDER')
path = os.path.join(folder, FILENAME)

with open(path, 'r') as file:
    first = file.readline(1)
    print(f"First row is: '{first}'")


## Example

As mentioned before, this statement is often used to create resources. These examples will open, read/write, close files & perform HTTP requests. 

### HTTP GET Requests

When using <code>aiohttp</code> module, each request is handled asynchronously & must use <font color='#bb9af7'>with</font> statement. Based on <a href='https://realpython.com/python-with-statement/'>Leodanis Pozo Ramos</a>' example.

In [ ]:
import asyncio
import os
import time

# py -m pip install aiofiles
# py -m pip install aiohttp
# py -m pip install tldextract
import aiofiles
import aiohttp
import dotenv
import tldextract

NS_2_MS = 1e6
NS_2_US = 1e3


dotenv.load_dotenv()
folder = os.getenv('TEMP_FOLDER')


def get_time(start: int, factor: int):
    return (time.time_ns() - start) / factor


def parse_url(url: str):
    return tldextract.extract(url).domain


async def get(url: str):
    start = time.time_ns()
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            print(f'{url}: status -> {response.status}')
            html = await response.text()
            print(f'{url}: time -> {get_time(start, NS_2_MS):.2f}ms')
            return html


async def writefile(path, doc):
    start = time.time_ns()
    async with aiofiles.open(path, mode='w', encoding='utf-8') as file:
        await file.write(doc)
        print(f'{os.path.basename(path)}: time -> {get_time(start, NS_2_US):.2f}us')


async def main():
    start = time.time_ns()
    urls = ['https://www.amazon.com', 'https://www.ebay.com']
    domains = [parse_url(url) for url in urls]
    requests = [get(url) for url in urls]

    documents = await asyncio.gather(*requests)
    tasks = [writefile(f'{folder}/{url}.html', doc) for url, doc in zip(domains, documents)]

    await asyncio.gather(*tasks)
    print(f'{main.__name__}: time -> {get_time(start, NS_2_MS):.2f}ms')


# Python: asyncio.run(main())
# Jupyter: await main()
await main()
